<h1>Preprocessing for Thesis Work</h1>

Author: Joshua White

This notebook will contain code used to preprocess a csv file for my CSCE 799 Expirment and my CSCE 623 Project. 

Sources:  
The following two articles I used to get me started  
https://towardsdatascience.com/the-basics-of-eda-with-candy-83b2e8ad9e63  
https://towardsdatascience.com/nlp-for-beginners-cleaning-preprocessing-text-data-ae8e306bef0f  
Useful source for pandas: https://www.geeksforgeeks.org/iterating-over-rows-and-columns-in-pandas-dataframe/  

First we will start with some Exploratory Data Analysis (EDA), or just looking at the data. In this notebook I will be looking at the 


In [1]:
#Imports:
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import string #Will be used for a list of punctuation
import re

#NLTK Imports:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

Now I'm going to use pandas to import the csv file into a data frame. 
*** IF YOU WANT TO CHANGE THE INPUT CSV FILENAME DO SO HERE *** The nyc-jobs.csv file will be what I'm initially working on and most comments in this notebook will reflect that. 

In [2]:
DFrame = pd.read_csv('nyc-jobs.csv')

<h2>Initial Look at the data</h2>

head() call just to peek at the top 5 rows of the file:

In [3]:
DFrame.head()

,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,...,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
0,87990,DEPARTMENT OF BUSINESS SERV.,Internal,1,Account Manager,CONTRACT REVIEWER (OFFICE OF L,40563,1,NaN,NaN,...,"Salary range for this position is: $42,405 - $...",NaN,NaN,NaN,NaN,New York City residency is generally required ...,2011-06-24T00:00:00.000,NaN,2011-06-24T00:00:00.000,2019-12-17T00:00:00.000
1,97899,DEPARTMENT OF BUSINESS SERV.,Internal,1,"EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT",ADMINISTRATIVE BUSINESS PROMOT,10009,M3,NaN,F,...,NaN,"In addition to applying through this website, ...",NaN,NaN,NaN,New York City residency is generally required ...,2012-01-26T00:00:00.000,NaN,2012-01-26T00:00:00.000,2019-12-17T00:00:00.000
2,132292,NYC HOUSING AUTHORITY,External,52,Maintenance Worker - Technical Services-Heatin...,MAINTENANCE WORKER,90698,0,Maintenance & Operations,F,...,1. A Motor Vehicle Driverâ€™s License valid i...,"Click the ""Apply Now"" button.",NaN,NaN,NaN,NYCHA has no residency requirements.,2013-10-24T00:00:00.000,NaN,2013-12-12T00:00:00.000,2019-12-17T00:00:00.000
3,132292,NYC HOUSING AUTHORITY,Internal,52,Maintenance Worker - Technical Services-Heatin...,MAINTENANCE WORKER,90698,0,Maintenance & Operations,F,...,1. A Motor Vehicle Driverâ€™s License valid i...,"Click the ""Apply Now"" button.",NaN,NaN,NaN,NYCHA has no residency requirements.,2013-10-24T00:00:00.000,NaN,2013-12-12T00:00:00.000,2019-12-17T00:00:00.000
4,133921,NYC HOUSING AUTHORITY,Internal,50,Temporary Painter,PAINTER,91830,0,Maintenance & Operations,F,...,SPECIAL NOTE: 1. This is a temporary assig...,"Click the ""Apply Now"" button.",NaN,NaN,NaN,NYCHA has no residency requirement.,2014-01-09T00:00:00.000,NaN,2014-01-08T00:00:00.000,2019-12-17T00:00:00.000


shape call to look at the size of the rows & column of the data frame object:

In [4]:
DFrame.shape

(2946, 28)

<h2>Check for Missing Values</h2>

Now if we want to we can check to see how many missing values we have per row. To do this we will use the isnull() function on the data frame, then use .mean() to get the percent of missing values from each column, and finally add .sort_values(ascending=False) to sort the list (or you could leave it off if you wanted).

In [5]:
DFrame.isnull().mean().sort_values(ascending=False)
#Flip the comment if you want to sort the values or not
#DFrame.isnull().mean()

Recruitment Contact              1.000000
Post Until                       0.704345
Hours/Shift                      0.699932
Work Location 1                  0.539036
Additional Information           0.370672
Preferred Skills                 0.133401
Full-Time/Part-Time indicator    0.066191
Minimum Qual Requirements        0.006789
Posting Updated                  0.001358
Process Date                     0.001358
Residency Requirement            0.001358
Posting Date                     0.001358
Job Category                     0.000679
To Apply                         0.000339
Title Code No                    0.000000
Agency                           0.000000
Posting Type                     0.000000
# Of Positions                   0.000000
Business Title                   0.000000
Civil Service Title              0.000000
Work Location                    0.000000
Level                            0.000000
Salary Range From                0.000000
Salary Range To                  0

<h2> Remove Columns</h2>

After looking at the above list you can see if you want to remove columns or not. In the nyc-jobs.csv file the "Recruitment Contact" column was 100% empty, so we will just remove it using .drop(columns=['column name here'], inplace=True). Note that this just removes the column from the data frame object, as of right now the nyc-jobs.csv file is still 28 columns!

In [6]:
#If you wanted to look at the contents of the columns that arn't NaN (empty) we could with the following line
#DFrame['Recruitment Contact'].value_counts(dropna=False)

In [7]:
DFrame.drop(columns=['Recruitment Contact'], inplace=True)
#Check to see that it was removed using shape
DFrame.shape

(2946, 27)

We can use the .dtypes to see the data type for each column. When you see object as the type it is normally a string. 

In [8]:
DFrame.dtypes

Job ID                             int64
Agency                            object
Posting Type                      object
# Of Positions                     int64
Business Title                    object
Civil Service Title               object
Title Code No                     object
Level                             object
Job Category                      object
Full-Time/Part-Time indicator     object
Salary Range From                float64
Salary Range To                  float64
Salary Frequency                  object
Work Location                     object
Division/Work Unit                object
Job Description                   object
Minimum Qual Requirements         object
Preferred Skills                  object
Additional Information            object
To Apply                          object
Hours/Shift                       object
Work Location 1                   object
Residency Requirement             object
Posting Date                      object
Post Until      

<h2>Fill NaNs</h2>

We have seen how much of our data frame is empty, or NaN, now we can use a for loop that will replace all of the missing values with a string "unknown" or a 0 for our int64 and float64 types. 

In [9]:
for col in DFrame:
    if type(DFrame[col])=='object':
        #Fill all emplty values with the 'unknown', or can change here
        DFrame[col] = DFrame[col].fillna(value='unknown')
    else:
        #Here we will change all empty values with a 0
        DFrame[col] = DFrame[col].fillna(value=0)

Then check to make sure it worked

In [10]:
DFrame.isnull().sum().sum()

0

<h2>Normalize Column Names</h2>

Now we can change the column names to be lowercase and void of special characters. To look at a list of the column names use .columns on the data frame object. 

In [11]:
DFrame.columns

Index(['Job ID', 'Agency', 'Posting Type', '# Of Positions', 'Business Title',
       'Civil Service Title', 'Title Code No', 'Level', 'Job Category',
       'Full-Time/Part-Time indicator', 'Salary Range From', 'Salary Range To',
       'Salary Frequency', 'Work Location', 'Division/Work Unit',
       'Job Description', 'Minimum Qual Requirements', 'Preferred Skills',
       'Additional Information', 'To Apply', 'Hours/Shift', 'Work Location 1',
       'Residency Requirement', 'Posting Date', 'Post Until',
       'Posting Updated', 'Process Date'],
      dtype='object')

Then we will use .lower() and .str.replace() to make any changes.

In [12]:
DFrame.columns = [x.lower() for x in DFrame.columns.str.replace(" ","_").str.replace("/","_").str.replace("#","no")]
DFrame.columns

Index(['job_id', 'agency', 'posting_type', 'no_of_positions', 'business_title',
       'civil_service_title', 'title_code_no', 'level', 'job_category',
       'full-time_part-time_indicator', 'salary_range_from', 'salary_range_to',
       'salary_frequency', 'work_location', 'division_work_unit',
       'job_description', 'minimum_qual_requirements', 'preferred_skills',
       'additional_information', 'to_apply', 'hours_shift', 'work_location_1',
       'residency_requirement', 'posting_date', 'post_until',
       'posting_updated', 'process_date'],
      dtype='object')

<h2>Next Steps</h2>

Now we are done with the EDA steps for the data frame and we will move on to other preprocessing. 

<h2>Remove Duplicated Lines</h2>

Upon looking through the nyc-jobs.csv file there are some lines are are duplicates, and an easy way to tell is based off of the 'job_id' column value. You can use pandas to remove the duplicated lines. 

Source:
https://stackoverflow.com/questions/15741564/removing-duplicate-rows-from-a-csv-file-using-a-python-script &
https://www.geeksforgeeks.org/python-pandas-dataframe-drop_duplicates/


In [13]:
DFrame.drop_duplicates(subset = "job_id", inplace = True)

If you look at the origional .head() call lines 2 and 3 had the same "job_id", we can call .head() again to check to ensure the duplicaiton removal worked. 

In [14]:
DFrame.head()

,job_id,agency,posting_type,no_of_positions,business_title,civil_service_title,title_code_no,level,job_category,full-time_part-time_indicator,...,preferred_skills,additional_information,to_apply,hours_shift,work_location_1,residency_requirement,posting_date,post_until,posting_updated,process_date
0,87990,DEPARTMENT OF BUSINESS SERV.,Internal,1,Account Manager,CONTRACT REVIEWER (OFFICE OF L,40563,1,0,0,...,â€¢\tExcellent interpersonal and organizationa...,"Salary range for this position is: $42,405 - $...",0,0,0,New York City residency is generally required ...,2011-06-24T00:00:00.000,0,2011-06-24T00:00:00.000,2019-12-17T00:00:00.000
1,97899,DEPARTMENT OF BUSINESS SERV.,Internal,1,"EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT",ADMINISTRATIVE BUSINESS PROMOT,10009,M3,0,F,...,0,0,"In addition to applying through this website, ...",0,0,New York City residency is generally required ...,2012-01-26T00:00:00.000,0,2012-01-26T00:00:00.000,2019-12-17T00:00:00.000
2,132292,NYC HOUSING AUTHORITY,External,52,Maintenance Worker - Technical Services-Heatin...,MAINTENANCE WORKER,90698,0,Maintenance & Operations,F,...,1. A High School Diploma or GED. 2. CDL Dri...,1. A Motor Vehicle Driverâ€™s License valid i...,"Click the ""Apply Now"" button.",0,0,NYCHA has no residency requirements.,2013-10-24T00:00:00.000,0,2013-12-12T00:00:00.000,2019-12-17T00:00:00.000
4,133921,NYC HOUSING AUTHORITY,Internal,50,Temporary Painter,PAINTER,91830,0,Maintenance & Operations,F,...,0,SPECIAL NOTE: 1. This is a temporary assig...,"Click the ""Apply Now"" button.",0,0,NYCHA has no residency requirement.,2014-01-09T00:00:00.000,0,2014-01-08T00:00:00.000,2019-12-17T00:00:00.000
6,137433,DEPT OF HEALTH/MENTAL HYGIENE,Internal,1,Contract Analyst,PROCUREMENT ANALYST,12158,3,"Finance, Accounting, & Procurement",F,...,Strong analytical background; advanced profic...,0,Apply online with a cover letter to https://a1...,0,"42-09 28th Street, Long Island City, NY",New York City residency is generally required ...,2013-12-09T00:00:00.000,0,2013-12-09T00:00:00.000,2019-12-17T00:00:00.000


In [15]:
DFrame.shape

(1661, 27)

As you can see from the new head call the duplicate line was removed and we are down to 1661 rows of data. 

<h2>Combining Columns in the Data Frame</h2>  

For right now we want to combine the 'business_title' and 'job_description' columns into a new column called 'processed_text'. You can do so in pandas with the following code. 

In [16]:
DFrame['processed_text'] = DFrame['business_title'] + " " + DFrame['job_description']

<h2>Remove HTML</h2>

I could not find any HTML tags in the nyc-jobs.csv file, but if there were we could remove them from a column with the following code. 

In [17]:
#First I will just define the function
def remove_html(text):
    soup = BeautifulSoup(text, 'lxml')
    html_free = soup.get_text()
    return html_free

#Now lets say we wanted to run it on the 'job_description' column you could uncomment this line
#DFrame['processed_text'] = DFrame['processed_text'].apply(lambda x: remove_html(x))
#And could review the changes with this line
#DFrame['processed_text'].head(10)

<h2>Remove Punctuation</h2>

Now we can remove punctuation from a column, treating it as a Series, and keep everything that is not in string.punctuation, which is a list of all punctuation that imported with the import string at the begining. 

For now I will only do this on the 'processed_text' column, but it will be easy to modify in the future if necessary.

One thing of note about using this is that this method does not account for typos, e.g. "however,the" will turn into "howeverthe". If necessary we can create a copy of all columns we perform this on and just run any models on the columns seperatly if we are finding weird quirks in the data. 

In [18]:
#First I will just define the function
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    # This line removes all numbers and non-chars
    # source: https://kavita-ganesan.com/extracting-keywords-from-text-tfidf/#.XpRvAVNKi8U 
    no_punct = re.sub("(\\d|\\W)+"," ",no_punct)
    return no_punct

#Now I will just call it on the 'job_description' column and call .head() on it to make sure its not there
DFrame['processed_text'] = DFrame['processed_text'].apply(lambda x: remove_punctuation(x))
DFrame['processed_text'].head(10)

0     Account Manager Division of Economic Financial...
1     EXECUTIVE DIRECTOR BUSINESS DEVELOPMENT The Ne...
2     Maintenance Worker Technical ServicesHeating U...
4     Temporary Painter Responsibilities of selected...
6     Contract Analyst OPEN TO PERMANENT PROCUREMENT...
7     Associate Chemist Working in the Distribution ...
8     Cost Estimating Manager Reporting to the Deput...
9     Office Manager Performs essential administrati...
10    Deputy Director Audits and Accounts DoITT prov...
11    CERTIFIED IT ADMINISTRATOR WAN Level Oversees ...
Name: processed_text, dtype: object

<h2>Tokenize</h2>

Now we will tokenize the text by column. This is when we break up the strings into a list of words or pieces using RegEx. The pattern also removes punctuation, and we can call .lower() in the lambda to make everything lowercase at this step as well. 

The RegexpTokenizer man page: https://kite.com/python/docs/nltk.RegexpTokenizer
The \w here is: When the LOCALE and UNICODE flags are not specified, matches any alphanumeric character and the underscore; this is equivalent to the set [a-zA-Z0-9_]. With LOCALE, it will match the set [0-9_] plus whatever characters are defined as alphanumeric for the current locale. If UNICODE is set, this will match the characters [0-9_] plus whatever is classified as alphanumeric in the Unicode character properties database.
Source: https://docs.python.org/2/library/re.html

Some other RegEx patterns we could use:
‘\w+|\$[\d\.]+|\S+’ = splits up by spaces or by periods that are not attached to a digit
‘\s+’, gaps=True = grabs everything except spaces as a token
‘[A-Z]\w+’ = only words that begin with a capital letter
source: https://towardsdatascience.com/nlp-for-beginners-cleaning-preprocessing-text-data-ae8e306bef0f

In [19]:
#Create the tokenizer
tokenizer = RegexpTokenizer(r'\w+')

#Now call .tokenize on the 'job_description' column. We could extend this to more columns as necessary
DFrame['processed_text'] = DFrame['processed_text'].apply(lambda x: tokenizer.tokenize(x.lower()))
#And lets look at the output of this
DFrame['processed_text'].head(10)

0     [account, manager, division, of, economic, fin...
1     [executive, director, business, development, t...
2     [maintenance, worker, technical, servicesheati...
4     [temporary, painter, responsibilities, of, sel...
6     [contract, analyst, open, to, permanent, procu...
7     [associate, chemist, working, in, the, distrib...
8     [cost, estimating, manager, reporting, to, the...
9     [office, manager, performs, essential, adminis...
10    [deputy, director, audits, and, accounts, doit...
11    [certified, it, administrator, wan, level, ove...
Name: processed_text, dtype: object

<h2>Remove Stop Words</h2>

We will use a stop word list from the nltk.corpus, but we could use others if we wnated to by changeing the function we are about to define here. If you would like to see a list of all the stop words you could run stopwords.word('english'), or in any other language supported by NLTK.
**This step takes a while to run**

In [20]:
#First I will define the function to remove the stopwords
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

#Now we will just do it on the 'job_description' column for now, but could extend this to other columns easily
DFrame['processed_text'] = DFrame['processed_text'].apply(lambda x: remove_stopwords(x))
DFrame['processed_text'].head(10)

0     [account, manager, division, economic, financi...
1     [executive, director, business, development, n...
2     [maintenance, worker, technical, servicesheati...
4     [temporary, painter, responsibilities, selecte...
6     [contract, analyst, open, permanent, procureme...
7     [associate, chemist, working, distribution, wa...
8     [cost, estimating, manager, reporting, deputy,...
9     [office, manager, performs, essential, adminis...
10    [deputy, director, audits, accounts, doitt, pr...
11    [certified, administrator, wan, level, oversee...
Name: processed_text, dtype: object

<h2>Stemming &(or) Lemmatization</h2>

Here we can either perform stemming or lemmatization on our list of words, but because we will be performing keyword extraction and then using the keywords as input to ConceptNet I will perform lemmatization on the 'job_description' column because stemming can return strings that are not actually words and we want to ensure a keyword gets a hit in ConceptNet. I will still include the function for stemming for future use (if necessary). 

In [21]:
#The code for lemmatization:
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text

DFrame['processed_text'] = DFrame['processed_text'].apply(lambda x: word_lemmatizer(x))
DFrame['processed_text'].head(10)

0     [account, manager, division, economic, financi...
1     [executive, director, business, development, n...
2     [maintenance, worker, technical, servicesheati...
4     [temporary, painter, responsibility, selected,...
6     [contract, analyst, open, permanent, procureme...
7     [associate, chemist, working, distribution, wa...
8     [cost, estimating, manager, reporting, deputy,...
9     [office, manager, performs, essential, adminis...
10    [deputy, director, audit, account, doitt, prov...
11    [certified, administrator, wan, level, oversee...
Name: processed_text, dtype: object

In [22]:
# The function for Stemming and then the line to call it commented out
stemmer = PorterStemmer()
def word_stemmer(text):
    stem_text = " ".join([stemmer.stem(i) for i in text])
    return stem_text

#DFrame['processed_text'] = DFrame['processed_text'].apply(lambda x: word_stemmer(x))
#DFrame['processed_text'].head(10)

<h2>Turning Lists back into strings</h2>
Right now our 'job description' column is a column of lists, but we want to turn it back into just a column of strings for the next steps. Below is the code to perform that. 

In [23]:
#Right now our 'job_description' column is a column of lists, and we just want it 
#to be a column of strings. We will transform it here
def convert_to_text(text):
    return ' '.join(text)

DFrame['processed_text'].head(10)

0     [account, manager, division, economic, financi...
1     [executive, director, business, development, n...
2     [maintenance, worker, technical, servicesheati...
4     [temporary, painter, responsibility, selected,...
6     [contract, analyst, open, permanent, procureme...
7     [associate, chemist, working, distribution, wa...
8     [cost, estimating, manager, reporting, deputy,...
9     [office, manager, performs, essential, adminis...
10    [deputy, director, audit, account, doitt, prov...
11    [certified, administrator, wan, level, oversee...
Name: processed_text, dtype: object

In [24]:
#Now we will make a new column called 'jd_text' and run the function on it
DFrame['processed_text'] = DFrame['processed_text'].apply(lambda x: convert_to_text(x))
DFrame['processed_text'].head(10)

0     account manager division economic financial op...
1     executive director business development new yo...
2     maintenance worker technical servicesheating u...
4     temporary painter responsibility selected cand...
6     contract analyst open permanent procurement an...
7     associate chemist working distribution water q...
8     cost estimating manager reporting deputy direc...
9     office manager performs essential administrati...
10    deputy director audit account doitt provides s...
11    certified administrator wan level oversees des...
Name: processed_text, dtype: object

<h2>Removing Non-English Words</h2>  

Currently there are some non-english words in our documents (they are either misspellings or words that were cut off somehow) that need to be filtered out. 

Source: https://stackoverflow.com/questions/41290028/removing-non-english-words-from-text-using-python

In [25]:
words = set(nltk.corpus.words.words())
def remove_non_english_words(text):
    return " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() in words or not w.isalpha())

DFrame['processed_text'] = DFrame['processed_text'].apply(lambda x: remove_non_english_words(x))
DFrame['processed_text'].head(10)

0     account manager division economic financial op...
1     executive director business development new yo...
2     maintenance worker technical unit direct super...
4     temporary painter responsibility selected cand...
6     contract analyst open permanent procurement an...
7     associate chemist working distribution water q...
8     cost manager deputy director technical latitud...
9     office manager essential administrative functi...
10    deputy director audit account sustained effici...
11    certified administrator wan level design insta...
Name: processed_text, dtype: object

<h2>Exporting the Data Frame</h2>

At this point if you would like to export the data frame to a csv file you could run the following code, just make sure you change the file path to the correct directory. 

Source: https://datatofish.com/export-dataframe-to-csv/

In [27]:
DFrame.to_csv(r'C:\Users\Joshua\Google Drive\Thesis Work\Python\nyc-jobs-cleaned.csv', index = False, header = True)